In [36]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import numpy as np
import os
import json
import math
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import gc

# === 激活 DirectML (Windows AMD) ===
try:
    import torch_directml
    DEVICE = torch_directml.device()
    print(f"✅ 成功激活 DirectML 加速设备: {DEVICE}")
except ImportError:
    DEVICE = torch.device("cpu")
    print("❌ 未检测到 torch-directml，使用 CPU")

# ================= 配置区域 =================
# 1. 指向新数据集
DATA_PATH = "../output/chess_transformer4/dataset_seq_strict.csv" # 确保指向最新的清洗版数据
# 2. 指向一个新的空文件夹，用于保存第二阶段的训练成果
CHECKPOINT_DIR = "../output/chess_transformer4/checkpoints_dual"  # 改个目录名，区分单输出版本
MODEL_NAME = "chess_transformer_dual.pth"
VOCAB_FILE = "vocab.json"
# 3. 原始模型的路径 (用于读取权重)
PRETRAINED_PATH = "../output/chess_transformer/checkpoints_dual/chess_transformer_dual.pth"
OLD_VOCAB_PATH = "../output/chess_transformer/checkpoints_dual/vocab.json"

CHUNK_SIZE = 200000    
BATCH_SIZE = 512       
D_MODEL = 256
N_HEAD = 8
NUM_LAYERS = 4
LEARNING_RATE = 1e-4
MAX_SEQ_LEN = 40
EPOCHS = 10


✅ 成功激活 DirectML 加速设备: privateuseone:0


In [37]:

# ================= 1. 词表与数据处理 =================

class ChessVocab:
    def __init__(self):
        self.move2id = {"<PAD>": 0, "<UNK>": 1}
        self.id2move = {0: "<PAD>", 1: "<UNK>"}
        
    def build_from_file(self, csv_path):
        print("🚀 正在扫描大文件构建词表...")
        unique_moves = set()
        chunk_iter = pd.read_csv(csv_path, chunksize=CHUNK_SIZE)
        for i, chunk in enumerate(chunk_iter):
            print(f"  扫描 Chunk {i+1}...", end='\r')
            chunk = chunk.dropna(subset=['Moves'])
            moves_list = chunk['Moves'].str.split()
            for moves in moves_list:
                unique_moves.update(moves)
        
        print(f"\n✅ 扫描完成！发现 {len(unique_moves)} 种唯一棋步")
        for move in sorted(list(unique_moves)):
            if move not in self.move2id:
                idx = len(self.move2id)
                self.move2id[move] = idx
                self.id2move[idx] = move
        
    def encode(self, moves_list):
        return [self.move2id.get(m, 1) for m in moves_list]

    def save(self, path):
        with open(path, 'w') as f:
            json.dump(self.move2id, f)

    def load(self, path):
        with open(path, 'r') as f:
            self.move2id = json.load(f)
        self.id2move = {v: k for k, v in self.move2id.items()}

# === 修改点 1: Dataset 读取双目标 ===
class ChessDataset(Dataset):
    def __init__(self, df, vocab, max_len=40):
        self.vocab = vocab
        self.max_len = max_len
        self.data = []
        self.targets = []
        
        # 同时读取 White 和 Black 的 ELO
        w_elos = df['WhiteElo'].values.astype(np.float32)
        b_elos = df['BlackElo'].values.astype(np.float32)
        moves_series = df['Moves'].str.split()
        
        for moves, w, b in zip(moves_series, w_elos, b_elos):
            moves = moves[:max_len]
            token_ids = self.vocab.encode(moves)
            self.data.append(torch.tensor(token_ids, dtype=torch.long))
            # Target 变成了 [white_elo, black_elo] 的列表
            self.targets.append([w, b])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

# === 修改点 2: Collate 函数适配双目标形状 ===
def collate_fn(batch):
    inputs, targets = zip(*batch)
    inputs_padded = pad_sequence(inputs, batch_first=True, padding_value=0)
    
    # targets 现在是 list of lists [[w1, b1], [w2, b2], ...]
    # 转换为 tensor shape [batch_size, 2]
    targets_tensor = torch.tensor(targets, dtype=torch.float32)
    
    padding_mask = (inputs_padded == 0)
    return inputs_padded, targets_tensor, padding_mask


In [38]:

# ================= 2. 模型定义 =================

class ChessTransformer(nn.Module):
    def __init__(self, vocab_size, d_model, nhead, num_layers, max_len=40):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model, padding_idx=0)
        self.pos_embedding = nn.Parameter(torch.randn(1, max_len, d_model))
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=d_model*4, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # === 修改点 3: 输出层改为 2 ===
        self.fc = nn.Sequential(
            nn.Linear(d_model, d_model // 2),
            nn.ReLU(),
            nn.Linear(d_model // 2, 2) # Output: [White_ELO, Black_ELO]
        )

    def forward(self, x, src_key_padding_mask):
        seq_len = x.size(1)
        x = self.embedding(x) + self.pos_embedding[:, :seq_len, :]
        x = self.transformer(x, src_key_padding_mask=src_key_padding_mask)
        x = x.mean(dim=1) 
        out = self.fc(x)
        return out


In [39]:

# ================= 3. 工具函数 =================

def save_checkpoint(model, optimizer, epoch, chunk_idx, loss, vocab, dir_path):
    if not os.path.exists(dir_path): os.makedirs(dir_path)
    ckpt_path = os.path.join(dir_path, MODEL_NAME)
    
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
        # === 新增：保存当前已完成的 Chunk 索引 ===
        'last_completed_chunk': chunk_idx 
    }, ckpt_path)
    
    vocab.save(os.path.join(dir_path, VOCAB_FILE))

def load_checkpoint(model, optimizer, dir_path):
    ckpt_path = os.path.join(dir_path, MODEL_NAME)
    vocab_path = os.path.join(dir_path, VOCAB_FILE)
    
    start_epoch = 0
    start_chunk = -1 # === 新增：默认为 -1，表示没跑过任何块
    vocab = ChessVocab()
    
    if os.path.exists(ckpt_path) and os.path.exists(vocab_path):
        print(f"🔄 正在加载存档: {ckpt_path}")
        try:
            # 强制 map_location='cpu' 避免 AMD 显卡报错
            checkpoint = torch.load(ckpt_path, map_location='cpu')
            
            model.load_state_dict(checkpoint['model_state_dict'])
            if optimizer:
                optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
                
            start_epoch = checkpoint['epoch']
            
            # === 新增：尝试读取上次中断的块索引 ===
            start_chunk = checkpoint.get('last_completed_chunk', -1)
            
            print(f"📅 恢复进度: 第 {start_epoch+1} 轮, 第 {start_chunk+1} 个块")
            
        except Exception as e:
            print(f"❌ 存档加载失败: {e}")
            return 0, -1, vocab # 修改返回值

        vocab.load(vocab_path)
    else:
        print("✨ 开始新训练")
    
    # === 修改：返回 start_epoch 和 start_chunk ===
    return start_epoch, start_chunk, vocab

# Python 版严格清洗 (必须与 C++ 一致)
def clean_strict(move_str):
    raw = move_str
    if raw.startswith("O-O"):
        if raw.startswith("O-O-O"): return "O-O-O"
        return "O-O"
    rank_idx = -1
    for i in range(len(raw) - 1, -1, -1):
        if raw[i].isdigit():
            rank_idx = i
            break
    if rank_idx <= 0: return ""
    dest = raw[rank_idx-1 : rank_idx+1]
    first = raw[0]
    piece = ""
    if first.isupper(): piece = first
    promotion = ""
    if "=" in raw:
        idx = raw.find("=")
        if idx + 1 < len(raw): promotion = "=" + raw[idx+1]
    return piece + dest + promotion


In [40]:
def load_transfer_model(model, optimizer, pretrained_path, old_vocab_path):
    # 1. 必须强制使用旧词表！
    # 如果你对新数据集重新构建词表，ID会乱套 (比如旧词表里 e4=5，新词表里 e4=10)，模型就废了。
    vocab = ChessVocab()
    if os.path.exists(old_vocab_path):
        print(f"📚 正在加载原始词表: {old_vocab_path}")
        vocab.load(old_vocab_path)
    else:
        print("❌ 致命错误：找不到旧词表！无法进行迁移学习。")
        return 0, -1, None

    # 2. 加载旧模型权重
    if os.path.exists(pretrained_path):
        print(f"🚀 正在加载预训练模型权重: {pretrained_path}")
        # 强制 cpu 加载避免 DirectML 报错
        checkpoint = torch.load(pretrained_path, map_location='cpu')
        
        try:
            model.load_state_dict(checkpoint['model_state_dict'])
        except RuntimeError as e:
            print(f"❌ 权重形状不匹配: {e}")
            return 0, -1, vocab

        # 3. 关键点：我们【不】加载 optimizer 和 epoch/chunk
        # 我们希望在新数据上从头开始微调，所以进度重置为 0
        start_epoch = 0
        start_chunk = -1
        
        print("✅ 权重加载成功！进度已重置，准备在新数据集上开始训练。")
    else:
        print("✨ 未找到预训练模型，将从头开始随机初始化训练。")
        start_epoch = 0
        start_chunk = -1

    return start_epoch, start_chunk, vocab

In [41]:
# ================= 4. 训练逻辑 (修正版) =================

def train_dual():
    if not os.path.exists(DATA_PATH):
        print("❌ 找不到数据文件")
        return

    # 预加载词表对象（稍后使用）
    vocab = ChessVocab()
    
    # ---------------------------------------------------------
    # 分支 1: 检测到 Phase 2 存档 (断点续训)
    # ---------------------------------------------------------
    if os.path.exists(os.path.join(CHECKPOINT_DIR, MODEL_NAME)):
        print("🔄 检测到 Phase 2 存档，准备继续训练...")
        
        # 【关键修正】先加载 Phase 2 的词表，获取正确的大小！
        current_vocab_path = os.path.join(CHECKPOINT_DIR, VOCAB_FILE)
        if os.path.exists(current_vocab_path):
            vocab.load(current_vocab_path)
            real_vocab_size = len(vocab.move2id)
            print(f"📖 读取到现有词表，大小: {real_vocab_size}")
        else:
            print("❌ 严重错误：有模型存档但没有词表！无法恢复。")
            return

        # 使用正确的大小初始化模型
        model = ChessTransformer(real_vocab_size, D_MODEL, N_HEAD, NUM_LAYERS).to(DEVICE)
        optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, foreach=False)
        
        # 现在加载权重，就不会报错了
        start_epoch, start_chunk, vocab = load_checkpoint(model, optimizer, CHECKPOINT_DIR)

    # ---------------------------------------------------------
    # 分支 2: 没有 Phase 2 存档 (开始新的迁移学习)
    # ---------------------------------------------------------
    else:
        print("🌟 开始迁移学习 (Transfer Learning)...")
        # 加载旧词表 (Phase 1)
        if os.path.exists(OLD_VOCAB_PATH):
            vocab.load(OLD_VOCAB_PATH)
        else:
            # 如果没有旧词表，就根据新数据构建 (不推荐，但在 Phase 1 应该构建过)
            vocab.build_from_file(DATA_PATH)
        
        real_vocab_size = len(vocab.move2id)
        
        # 用旧词表大小初始化模型
        model = ChessTransformer(real_vocab_size, D_MODEL, N_HEAD, NUM_LAYERS).to(DEVICE)
        optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, foreach=False)
        
        # 加载预训练权重，重置进度
        start_epoch, start_chunk, vocab = load_transfer_model(model, optimizer, PRETRAINED_PATH, OLD_VOCAB_PATH)
        
        # 必须把旧词表保存到新目录，方便以后使用
        if not os.path.exists(CHECKPOINT_DIR): os.makedirs(CHECKPOINT_DIR)
        vocab.save(os.path.join(CHECKPOINT_DIR, VOCAB_FILE))

    # === 公共训练逻辑 ===
    criterion = nn.MSELoss()

    for epoch in range(start_epoch, EPOCHS):
        print(f"\n=== Epoch {epoch + 1}/{EPOCHS} ===")
        chunk_iterator = pd.read_csv(DATA_PATH, chunksize=CHUNK_SIZE)
        
        total_epoch_loss = 0
        chunks_count = 0
        
        for i, chunk in enumerate(chunk_iterator):
            # 快进逻辑
            if epoch == start_epoch and i <= start_chunk:
                if i % 10 == 0:
                    print(f"⏩ 跳过已训练数据块: {i}...", end='\r')
                continue
            
            chunk = chunk.dropna(subset=['Moves'])
            if len(chunk) == 0: continue
            
            train_df, val_df = train_test_split(chunk, test_size=0.05)
            
            train_dataset = ChessDataset(train_df, vocab)
            # 你的 12600KF 核多，这里 num_workers 可以试着设为 0 (最稳) 或 2
            train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn, num_workers=0)
            
            model.train()
            chunk_loss = 0
            pbar = tqdm(train_loader, desc=f"Ep {epoch+1} | Chunk {i+1}", leave=False)
            
            for inputs, targets, mask in pbar:
                inputs, targets, mask = inputs.to(DEVICE), targets.to(DEVICE), mask.to(DEVICE)
                
                optimizer.zero_grad()
                outputs = model(inputs, mask) 
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()
                
                chunk_loss += loss.item()
                pbar.set_postfix({'rmse': math.sqrt(loss.item())})
            
            avg_chunk_loss = chunk_loss / len(train_loader)
            total_epoch_loss += avg_chunk_loss
            chunks_count += 1
            
            del train_dataset, train_loader, train_df, val_df, chunk
            gc.collect() 
            
            # 每 5 个 chunk 保存一次
            if (i + 1) % 5 == 0:
                save_checkpoint(model, optimizer, epoch, i, avg_chunk_loss, vocab, CHECKPOINT_DIR)
                print(f"  [保存] Chunk {i+1} 完成")
        
        start_chunk = -1
        avg_epoch_loss = total_epoch_loss / chunks_count if chunks_count > 0 else 0
        print(f"Epoch {epoch+1} 完成. 平均 RMSE: {math.sqrt(avg_epoch_loss):.2f}")
        save_checkpoint(model, optimizer, epoch + 1, -1, avg_epoch_loss, vocab, CHECKPOINT_DIR)

In [42]:
# ================= 5. 预测接口 =================
# DEVICE = 'cpu'
# ================= 5. 预测接口 (修正版) =================

def predict_dual(raw_pgn):
    # 1. 路径检查
    # vocab_path = os.path.join(CHECKPOINT_DIR, OLD_VOCAB_PATH)
    vocab_path = OLD_VOCAB_PATH
    if not os.path.exists(vocab_path):
        print(f"❌ 找不到词表文件: {vocab_path}")
        print("请检查 CHECKPOINT_DIR 是否指向了正确的训练目录")
        return

    # 2. 【关键修正】先加载词表！
    vocab = ChessVocab()
    vocab.load(vocab_path)
    real_vocab_size = len(vocab.move2id)
    print(f"📖 加载词表成功，大小: {real_vocab_size}")
    
    # 3. 【关键修正】使用加载后的词表大小来初始化模型
    model = ChessTransformer(real_vocab_size, D_MODEL, N_HEAD, NUM_LAYERS).to(DEVICE)
    
    # 4. 加载权重 (此时模型结构已经正确，可以塞入权重了)
    # 这里的 load_checkpoint 主要是为了读权重，它内部也会读一次 vocab，但不影响
    ckpt_epoch, _, _ = load_checkpoint(model, None, CHECKPOINT_DIR)
    
    if ckpt_epoch == 0:
        print("⚠️ 警告: 似乎没有加载到有效的模型权重")

    model.eval() # 切换到预测模式
    
    # 5. 数据清洗 + 编码
    raw_moves = raw_pgn.split()
    clean_moves = [clean_strict(m) for m in raw_moves]
    clean_moves = [m for m in clean_moves if m] # 过滤空串
    clean_moves = clean_moves[:MAX_SEQ_LEN]
    
    if not clean_moves:
        print("⚠️ 输入无效或为空")
        return

    # 6. 转换为 Tensor 并移至显卡
    token_ids = vocab.encode(clean_moves)
    tensor_in = torch.tensor(token_ids, dtype=torch.long).unsqueeze(0).to(DEVICE)
    mask = torch.zeros((1, len(token_ids)), dtype=torch.bool).to(DEVICE)
    
    # 7. 推理
    print("-" * 30)
    print(f"输入对局: {' '.join(clean_moves[:10])}...")
    
    with torch.no_grad():
        preds = model(tensor_in, mask) # [1, 2]
        w_pred = preds[0][0].item()
        b_pred = preds[0][1].item()
        
        print(f"⚪ 白方预测 ELO: {w_pred:.0f}")
        print(f"⚫ 黑方预测 ELO: {b_pred:.0f}")
        print(f"⚖️ 实力分差: {abs(w_pred - b_pred):.0f}")
        
    print("-" * 30)

In [47]:
train_dual() # 训练模式


🌟 开始迁移学习 (Transfer Learning)...
📚 正在加载原始词表: ../output/chess_transformer/checkpoints_dual/vocab.json
🚀 正在加载预训练模型权重: ../output/chess_transformer/checkpoints_dual/chess_transformer_dual.pth
✅ 权重加载成功！进度已重置，准备在新数据集上开始训练。

=== Epoch 1/10 ===


C:\Users\Administrator\AppData\Local\Temp\ipykernel_26984\4088458396.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(pretrained_path, map_locati

Epoch 1 完成. 平均 RMSE: 180.73

=== Epoch 2/10 ===


Epoch 2 完成. 平均 RMSE: 176.80

=== Epoch 3/10 ===


Epoch 3 完成. 平均 RMSE: 174.50

=== Epoch 4/10 ===


...

Epoch 6 完成. 平均 RMSE: 169.87

=== Epoch 7/10 ===


Epoch 7 完成. 平均 RMSE: 168.62

=== Epoch 8/10 ===


Epoch 8 完成. 平均 RMSE: 167.46

=== Epoch 9/10 ===


Epoch 9 完成. 平均 RMSE: 166.51

=== Epoch 10/10 ===


Epoch 10 完成. 平均 RMSE: 165.36


In [52]:
# 预测模式示例
DEVICE = "cpu"
test_game = """
1. d4 b5 2. c4 bxc4 3. Nf3 Bb7 4. Na3 d5 5. e3 e6 6. b3 cxb3 7. Qxb3 Bc6 8. Ne5 Bd7 9. Qb7 f6 10. Qxa8 Bb4+ 11. Bd2 Bxd2+ 12. Kxd2 fxe5 13. Rb1 Nc6 14. Qxd8+ Kxd8 15. Bb5 Nge7 16. Rhc1 exd4 17. Bxc6 dxe3+ 18. fxe3 Nxc6 19. Rxc6 Bxc6 20. Rb8+ Kd7 21. Rxh8 h6 22. Rh7 Kd6 23. Rxg7 Be8 24. Nc2 c5 25. h4 a5 26. g4 a4 27. h5 Ke5 28. g5 hxg5 29. h6 Kf6 30. Rg8 Bg6 31. Ne1 g4 32. Ng2 g3 33. Nf4 Be4 34. Rxg3 e5 35. Nxd5+ Bxd5 36. h7 1-0""" 
predict_dual(test_game)

📖 加载词表成功，大小: 436
🔄 正在加载存档: ../output/chess_transformer4/checkpoints_dual\chess_transformer_dual.pth


C:\Users\Administrator\AppData\Local\Temp\ipykernel_26984\4055380088.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location='cp

📅 恢复进度: 第 11 轮, 第 0 个块
------------------------------
输入对局: d4 b5 c4 c4 Nf3 Bb7 Na3 d5 e3 e6...
⚪ 白方预测 ELO: 1612
⚫ 黑方预测 ELO: 1496
⚖️ 实力分差: 116
------------------------------
